In [5]:
import pickle as pk
import dgl as d
import pandas as pd
import numpy as np
from datasets.dataset import FeatureLookup
from models.node2vec import Node2vecModel as Node2vec2

In [6]:
class a:
    def __init__(self, node2vec_dim, node2vec_length, node2vec_walk, node2vec_epochs, node2vec_batchsize):
        self.node2vec_dim = node2vec_dim
        self.node2vec_length = node2vec_length
        self.node2vec_walk = node2vec_walk
        self.node2vec_epochs = node2vec_epochs
        self.node2vec_batchsize = node2vec_batchsize
        self.slices = 10

In [7]:
def node2vec_embedings(graph, args):

    # config
    p = 1
    q = 0.5
    embedding_dim = args.node2vec_dim
    walk_length = args.node2vec_length
    num_walks = args.node2vec_walk
    epochs = args.node2vec_epochs
    batch_size = args.node2vec_batchsize
    device = 'gpu'

    # config
    trainer = Node2vec2(
        graph,
        embedding_dim = embedding_dim,
        walk_length = walk_length,
        p = p,
        q = q,
        num_walks = num_walks,
        # weight_name = 'w',
        device = device,
    )

    trainer.train(epochs = epochs, batch_size = batch_size, learning_rate = 1e-2)

    node_features = trainer.embedding()

    nodes = node_features.detach().to('cpu').numpy()

    return nodes

# node2vec 基于服务切分
def eraser2(datasets, args):
    label = None

    servg = d.graph([])

    serv_lookup = FeatureLookup()

    sfile = pd.read_csv('./datasets/data/WSDREAM/原始数据/wslist_table.csv')
    sfile = pd.DataFrame(sfile)
    slines = sfile.to_numpy()

    for i in range(5825):
        serv_lookup.register('Sid', i)

    for sre in slines[:, 4]:
        serv_lookup.register('SRE', sre)

    for spr in slines[:, 2]:
        serv_lookup.register('SPR', spr)

    for sas in slines[:, 6]:
        serv_lookup.register('SAS', sas)

    servg.add_nodes(len(serv_lookup))

    for line in slines:
        sid = line[0]
        sre = serv_lookup.query_id(line[4])
        if not servg.has_edges_between(sid, sre):
            servg.add_edges(sid, sre)

        sas = serv_lookup.query_id(line[6])
        if not servg.has_edges_between(sid, sas):
            servg.add_edges(sid, sas)

        spr = serv_lookup.query_id(line[2])
        if not servg.has_edges_between(sid, spr):
            servg.add_edges(sid, spr)

    servg = d.add_self_loop(servg)
    servg = d.to_bidirected(servg)

    # node2vec
    ans = None
    ans = np.array(node2vec_embedings(servg, args))
    ans = ans[: 5825]

    df = np.array(ans)
    pk.dump(df, open(f'./pretrain/item_embeds.pk', 'wb'))

    def k_mean(inputs, k):
        from sklearn.cluster import KMeans
        kmeans = KMeans(n_clusters = k)
        kmeans.fit(inputs)
        return kmeans.labels_

    label = k_mean(df, 10)
    pk.dump(label, open(f'./pretrain/item_based_{args.slices}.pk', 'wb'))
#
args = a(128, 10, 50, 150, 512)
eraser2(None, args)

100%|██████████| 150/150 [03:11<00:00,  1.28s/it]


2022-11-26 19:09:52  	Node2vec loss = 0.7851


In [8]:
# node2vec
def eraser(datasets, args):
    label = None
    userg = d.graph([])
    user_lookup = FeatureLookup()
    ufile = pd.read_csv('./datasets/data/WSDREAM/原始数据/userlist_table.csv')
    ufile = pd.DataFrame(ufile)
    ulines = ufile.to_numpy()

    for i in range(339):
        user_lookup.register('User', i)

    for ure in ulines[:, 2]:
        user_lookup.register('URE', ure)

    for uas in ulines[:, 4]:
        user_lookup.register('UAS', uas)

    userg.add_nodes(len(user_lookup))

    for line in ulines:
        uid = line[0]
        ure = user_lookup.query_id(line[2])

        if not userg.has_edges_between(uid, ure):
            userg.add_edges(uid, ure)

        uas = user_lookup.query_id(line[4])
        if not userg.has_edges_between(uid, uas):
            userg.add_edges(uid, uas)

    userg = d.add_self_loop(userg)
    userg = d.to_bidirected(userg)

    # node2vec
    ans = None
    ans = np.array(node2vec_embedings(userg, args))
    ans = ans[: 339]

    # 整理特征
    df = np.array(ans)

    def k_mean(inputs, k):
        from sklearn.cluster import KMeans
        kmeans = KMeans(n_clusters = k)
        kmeans.fit(inputs)
        return kmeans.labels_

    label = k_mean(df, 10)
    pk.dump(label, open(f'./pretrain/user_based_{args.slices}.pk', 'wb'))

# def __init__(self, node2vec_dim, node2vec_length, node2vec_walk, node2vec_epochs, node2vec_batchsize):
args = a(128, 20, 50, 1000, 32)
# eraser(None, args)

In [9]:
import torch
torch.cuda.is_available()

True